In [16]:
# Health Insurance Cost Prediction
# İnsanların kişisel sağlık verilerinden sağlık sigortası masraflarını tahmin etmeye yçnelişk bir çalışma
# Supervise bir veriseti
# Süreki verilerden oluştuğu için regresyon lkullandık

# Section 1: Veri Önişleme

In [17]:
# python libraries
import numpy as np
import pandas as pd

# ML libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error


In [18]:
data = pd.read_csv("C:/Users/HAZAL/OneDrive/Masaüstü/Projeler/health_insurance_cost_estimate/insurance.csv")

In [19]:
data.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [20]:
data.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


In [21]:
# Eksik veri içeren satırlar kaldırılır
data = data.dropna()

In [22]:
# Verilerin sütunlarda dengeli dağılıp dağılmadığını kontrol edeceğiz.(balance/unbalance)

# İşlem yapmak istediğimiz sütunların isimlerini bir liste olarak tanımlıyoruz:
column_names = ["age", "sex", "bmi", "children", "smoker", "region", "charges"]
# Her sütun için frekans istatistiklerini saklamak için bir sözlük oluşturuyoruz:
statistics = {}

# Bir döngü kullanarak her sütunun frekans istatistiklerini hesaplıyoruz:
# Bu döngü, her sütunun değerlerini alır, value_counts yöntemini kullanarak her bir değerin kaç kez tekrarlandığını hesaplar 
# ve sonuçları statistics adlı sözlüğe ekler.
for column_name in column_names:
    column_stats = data[column_name].value_counts()
    statistics[column_name] = column_stats

# Bu döngü, her sütunun adını ve ilgili frekans istatistiklerini ekrana yazdırır. Her sütunun sonuçları daha anlamlı bir şekilde gösterilir.
for column_name, column_stat in statistics.items():
    print(f"{column_name} kolonundaki değerlerin kaç defa tekrarladıkları:\n{column_stat}\n")

age kolonundaki değerlerin kaç defa tekrarladıkları:
18    69
19    68
50    29
51    29
47    29
46    29
45    29
20    29
48    29
52    29
22    28
49    28
54    28
53    28
21    28
26    28
24    28
25    28
28    28
27    28
23    28
43    27
29    27
30    27
41    27
42    27
44    27
31    27
40    27
32    26
33    26
56    26
34    26
55    26
57    26
37    25
59    25
58    25
36    25
38    25
35    25
39    25
61    23
60    23
63    23
62    23
64    22
Name: age, dtype: int64

sex kolonundaki değerlerin kaç defa tekrarladıkları:
male      676
female    662
Name: sex, dtype: int64

bmi kolonundaki değerlerin kaç defa tekrarladıkları:
32.300    13
28.310     9
30.495     8
30.875     8
31.350     8
          ..
46.200     1
23.800     1
44.770     1
32.120     1
30.970     1
Name: bmi, Length: 548, dtype: int64

children kolonundaki değerlerin kaç defa tekrarladıkları:
0    574
1    324
2    240
3    157
4     25
5     18
Name: children, dtype: int64

smoker kolonundak

In [23]:
# One hot encoding(dummy variable) ile kategorik verilerin nominal verilere dönüştürülmesi

# Kategorik sütunları dönüştürerek yeni sütunlar oluştur
data = pd.get_dummies(data, columns=['sex', 'smoker', 'region'], prefix=['sex', 'smoker', 'region'])
data.head()

# İhtiyaç duyulmayan sütunları düşür
data.drop(['sex_female', 'smoker_no'], axis=1, inplace=True)

data.head()

,age,bmi,children,charges,sex_male,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,0,1,0,0,0,1
1,18,33.770,1,1725.55230,1,0,0,0,1,0
2,28,33.000,3,4449.46200,1,0,0,0,1,0
3,33,22.705,0,21984.47061,1,0,0,1,0,0
4,32,28.880,0,3866.85520,1,0,0,1,0,0


In [24]:
# Section 2: Verileri Bağımlı/Bağımsız olarak makine öğrenmesi modellerini test etme

In [25]:
# Veri setini x(bağımsız=age,bmi, ...) ve y(bağımlı=charges) olarak 2'ye ayırma
# charges(masraf) değeri dışındaki age, bmi, sex gibi değişkenlere bakarak masrafı tahmin etmeye çalışacağız yani masraf değeri bbu değerlere bağımlıdır.
# Bu nedenle chages kolonu bağımlı diğer kolonlar bağımsız değişkenlerdir.

y = data['charges']  # Bağımlı değişken
x = data.drop(['charges'], axis=1)  # Bağımsız değişkenler


In [26]:
#Section 3: Makine öğrenmesi aşamasına hazırlık

# Verileri train ve test olarak 2'ye ayırıyoruz.
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=46)

# Verisetinde normalizasyon işlemlerini yapıyoruz.
# Yani aslında bütün değerleri aynı scala içinde yazmaya çalışıcaz. Hepsini 0 ve 1 içinde yazıyoruz. 
# Çünkü bazı değerler sadece rakamadan ibaret olurken bazıları bin veya milyon olabilir.
# Bu nedenle hepsini aynı scala içine koyuyoruz ki modelimiz daha başarılı olsun.

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)



In [27]:
# Section 4: Makine öğrenmesi ve Model Performansı
# Her bir regresyon modeli sırayla tanımlanır, eğitilir ve tahminler yapılır. 

# max_depth'i değiştirerek sonucta bir değişiklik yapabiliriz.

# Decision Tree Alg.
tree_regression = DecisionTreeRegressor(random_state=42, max_depth=2) #karar ağacını oluşturduk
tree_regression.fit(x_train, y_train) # modeli eğitiyoruz. Makine x_train'deki verilere bakarak y_train'i  öğrenecek
predict_tree_regression = tree_regression.predict(x_test) #makinenin öğrenip öğrenmediğini test etme aşamasıdır. x_test'ten y_test'i tahmin etme

# Random Foret Alg.
random_regression = RandomForestRegressor(n_estimators=100, max_depth=4, random_state=42) #ağaç sayısı=100
random_regression.fit(x_train, y_train)
predict_random_regression = random_regression.predict(x_test)

# Lasso Alg.
lassoReg = Lasso(alpha=2)
lassoReg.fit(x_train,y_train)
predict_lasso = lassoReg.predict(x_test)

# Elastik Alg.
elastic_reg = ElasticNet(random_state=0)
elastic_reg.fit(x_train,y_train)
predict_elastic = elastic_reg.predict(x_test)

# Ridge Alg.
ridge_reg = Ridge()
ridge_reg.fit(x_train,y_train)
predict_ridge = ridge_reg.predict(x_test)

In [28]:
# Sonuçlar


# Modelin ne kadar başarılı ne kadar hatalı olduğunu R Kare, Mean Squared Error gibi metriklerle değerlendiriyoruz.

# 5 algoritmaya ait tahmin değerlerini predicts adlı bir liste içinde tutuyoruz.
predicts = [predict_tree_regression, predict_random_regression, predict_lasso, predict_elastic, predict_ridge]
algorithm_names = ["Decision Tree Alg.", "Random Forest Alg.", "Lasso Alg.", "Elastik Alg.", "Ridge Alg."]

# Sonucları açıklama fonk.
#Bir fonk. yazıp tahmin parametresini alıyoruz  ve her tahminin sonuçlarını hesaplayıp bunları data listesine atıp datayı ekrana bastırıyorum.
def performance_calculate(predict): 
    mae = mean_absolute_error(y_test, predict)
    mse = mean_squared_error(y_test, predict)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, predict)
    
    data = [mae, mse, rmse, r2]
    return data

In [29]:
# Ekrana yazdırmak

# Bu listeleriş bir for içinde geziyorum ve her bir tahmin için performance_calculate fonk. çalışır. Sonuç olarak burdna bir liste alıyor. Listeyi dataya attık. Dataları da seriler diye bir değişkene ekliyoruz.
# Oluşturduğumuz serileriş son olarak pandas'ta bir dataframe oluşturup içine attık.

series  = []
metrics = ["Mean Absolute Error(MAE)", "Mean Squared Error(MSE)", "Root Mean Squared Error(RMSE)", "R2"]

for i in predicts:
    data = performance_calculate(i)
    series.append(data)
    
df = pd.DataFrame(data=series, index=algorithm_names, columns=metrics)
pd.set_option('display.colheader_justify', 'center')
print(df.to_string())

                    Mean Absolute Error(MAE)  Mean Squared Error(MSE)  Root Mean Squared Error(RMSE)     R2   
Decision Tree Alg.         3299.054220             2.383989e+07                 4882.611455           0.824404
Random Forest Alg.         2622.613819             1.812672e+07                 4257.549152           0.866485
Lasso Alg.                 4201.461867             3.645130e+07                 6037.491554           0.731513
Elastik Alg.               5000.126534             4.595553e+07                 6779.051012           0.661508
Ridge Alg.                 4202.696230             3.644396e+07                 6036.883183           0.731567


In [30]:
# ÇIKTILAR

# R Kare 0 ile 1 arasında çıkmalı. Bizim için R Kare değeri ne kadar büyükse o algoritma daha başarılıdr.
# Bir algoritmanın en iyi olması için R Kare değerleriniin büyük olmasını diğer değerlerin küçük olmasını bekleriz.